# **Homework on Credit Risk**
## **Building an Application Scoring Model**

### **General Information**
- **Date assigned:** December 2, 2025  
- **Soft deadline:** 23:59 MSK, December 15, 2025  
- **Hard deadline:** 23:59 MSK, December 18, 2025  
- **Submission:** send your work to  
  \texttt{maria.vorobyova.ser@gmail.com}  
  with the subject format:
  \[
  \text{HSE\_CS\_[track]\_FullName}
  \]
  Example:
  \[
  \text{HSE\_CS\_PAD\_IVANOV\_IVAN\_IVANOVICH}
  \]

---

### **Grading and Penalties**
Maximum score: **10 points**

Late penalty:
\[
\text{Final Score} = 10 - \text{days late}
\]

Submission **after** the hard deadline is **not accepted**.

Work must be completed **independently**.  
Similar solutions → **plagiarism** → score **0**.

---

### **Score Reduction If**
- no comments in the notebook
- unclear or poorly written code
- incorrect analysis and conclusions

---

### **Task**
Build a scoring model estimating the **probability of default** at the **credit application stage**.

Follow the provided notebook strictly and complete every block.

---

### **Dataset**
Based on Kaggle competition:
\[
\text{Give Me Some Credit}
\]

Data source:  
https://www.kaggle.com/competitions/GiveMeSomeCredit/data  

Data description:  
**Data Dictionary.xlsx**



# **Work assignment:**
**1.Explatory Data Analysis - (Task weight: 20%)**

**2.Creating additional variables - (Task weight: 10%)**

**3. Model building (A logistic regression must be built on the WoE variables.)- (Task weight: 50%)**

**4. Using methods to reduce class imbalance - (Task weight: 20%)**

# **Submitting results:**

* Submit homework via the Yandex form as a link to your GitHub, where all files and code (Python) will be.
* GitHub must be open and the code must be working, without errors.
* Name the repository using the template (HW_4_2025-FirstName_LastName).
* Link to the Yandex form: https://forms.yandex.ru/u/68eece24505690c23425594c

We wish you good luck!✌

# Additional explanations for the task

In [6]:
# from google.colab import drive
import json
import zipfile

import pandas as pd

In [ ]:
def load_dataset(from_kaggle:bool = False) -> pd.DataFrame:
  '''
  The function downloads data from the Kaggle website if from_kaggle=True is set.
  Otherwise, the archive is read from a local file (this method is convenient for those who don't have access to Kaggle).
  params:
      - from_kaggle - индикатор откуда скачивать данные (True - c сайта kaggle, False -  c локального файла)
  return:
      - pd.DataFrame

  '''
  if from_kaggle:
    # запросит разрешение к гугл диску, необходимо дать это разрешение
    drive.mount('/content/drive')
    # установим kaggle
    !pip install kaggle -q
    !mkdir ~/.kaggle
    # копируем kaggle.json (предварительно, необходимо сгенерить токен на
    # сайте kaggle и сохранить к себе на гугл диск) в папку ~/.kaggle/
    !cp "/content/drive/MyDrive/Colab Notebooks/config/kaggle.json" ~/.kaggle/
    !kaggle competitions download -c GiveMeSomeCredit
  else:
    !gdown 1MRYG6P6ScMTzTmXJlWdPdYQVNi04wLSm
  # распаковка архива
  zip_ref = zipfile.ZipFile('GiveMeSomeCredit.zip', 'r')
  zip_ref.extractall()
  zip_ref.close()
  df_train = pd.read_csv('cs-training.csv')
  df_test = pd.read_csv('cs-test.csv')
  return df_train, df_test

In [9]:
df_train, df_test = load_dataset(from_kaggle=False)

'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
df_train

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


# 1.Exploratory Data Analysis. Максимально - (20%-2 балла)

- 0 points if the task is not completed
- 1 point if statistics are calculated and there are logical graphs (important, USEFUL graphs), but no conclusions are drawn
- 2 points if statistics are calculated and there are graphs (important, USEFUL graphs) and CONCLUSIONS are drawn (important, that the conclusions are correct)

In [20]:
# Exploratory Data Analysis
import numpy as np
# Set matplotlib backend before importing pyplot
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Display basic information about the dataset
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"\nTraining set shape: {df_train.shape}")
print(f"Test set shape: {df_test.shape}")
print(f"\nColumn names:")
print(df_train.columns.tolist())

# Display first few rows
print("\n" + "=" * 80)
print("FIRST 5 ROWS")
print("=" * 80)
print(df_train.head())

# Display data types and missing values
print("\n" + "=" * 80)
print("DATA TYPES AND MISSING VALUES")
print("=" * 80)
missing_info = pd.DataFrame({
    'Column': df_train.columns,
    'Data Type': df_train.dtypes,
    'Non-Null Count': df_train.count(),
    'Null Count': df_train.isnull().sum(),
    'Null Percentage': (df_train.isnull().sum() / len(df_train) * 100).round(2)
})
print(missing_info)

# Basic statistics for numerical variables
print("\n" + "=" * 80)
print("DESCRIPTIVE STATISTICS")
print("=" * 80)
print(df_train.describe())

# Target variable distribution
print("\n" + "=" * 80)
print("TARGET VARIABLE DISTRIBUTION (SeriousDlqin2yrs)")
print("=" * 80)
target_dist = df_train['SeriousDlqin2yrs'].value_counts()
target_pct = df_train['SeriousDlqin2yrs'].value_counts(normalize=True) * 100
print(f"\nCounts:\n{target_dist}")
print(f"\nPercentages:\n{target_pct}")
print(f"\nClass imbalance ratio: {target_dist[0] / target_dist[1]:.2f}:1")

# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
target_dist.plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon'])
axes[0].set_title('Target Variable Distribution (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('SeriousDlqin2yrs (0=No Default, 1=Default)', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_xticklabels(['No Default', 'Default'], rotation=0)

target_pct.plot(kind='bar', ax=axes[1], color=['skyblue', 'salmon'])
axes[1].set_title('Target Variable Distribution (Percentages)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('SeriousDlqin2yrs (0=No Default, 1=Default)', fontsize=12)
axes[1].set_ylabel('Percentage (%)', fontsize=12)
axes[1].set_xticklabels(['No Default', 'Default'], rotation=0)
for i, v in enumerate(target_pct):
    axes[1].text(i, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

# Distribution of numerical variables
numerical_cols = df_train.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('SeriousDlqin2yrs')  # Remove target variable
numerical_cols.remove('Unnamed: 0')  # Remove index column

print("\n" + "=" * 80)
print("DISTRIBUTION OF NUMERICAL VARIABLES")
print("=" * 80)

# Create histograms for numerical variables
n_cols = 3
n_rows = (len(numerical_cols) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
axes = axes.flatten()

for idx, col in enumerate(numerical_cols):
    # Remove outliers for better visualization (using IQR method)
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data_clean = df_train[(df_train[col] >= lower_bound) & (df_train[col] <= upper_bound)][col]
    
    axes[idx].hist(data_clean, bins=50, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col}\n(Mean: {df_train[col].mean():.2f}, Median: {df_train[col].median():.2f})', 
                        fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Value', fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].grid(True, alpha=0.3)

# Hide unused subplots
for idx in range(len(numerical_cols), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Box plots for numerical variables (by target)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
axes = axes.flatten()

for idx, col in enumerate(numerical_cols):
    # Remove extreme outliers for visualization
    Q1 = df_train[col].quantile(0.05)
    Q3 = df_train[col].quantile(0.95)
    data_subset = df_train[(df_train[col] >= Q1) & (df_train[col] <= Q3)]
    
    sns.boxplot(data=data_subset, x='SeriousDlqin2yrs', y=col, ax=axes[idx])
    axes[idx].set_title(f'{col} by Default Status', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('SeriousDlqin2yrs', fontsize=10)
    axes[idx].set_ylabel('Value', fontsize=10)
    axes[idx].set_xticklabels(['No Default', 'Default'])

for idx in range(len(numerical_cols), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Correlation matrix
print("\n" + "=" * 80)
print("CORRELATION MATRIX")
print("=" * 80)
corr_matrix = df_train[numerical_cols + ['SeriousDlqin2yrs']].corr()
print(corr_matrix['SeriousDlqin2yrs'].sort_values(ascending=False))

# Visualize correlation matrix
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numerical Variables', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Missing values visualization
missing_cols = missing_info[missing_info['Null Count'] > 0]
if len(missing_cols) > 0:
    print("\n" + "=" * 80)
    print("MISSING VALUES ANALYSIS")
    print("=" * 80)
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    missing_cols_sorted = missing_cols.sort_values('Null Count', ascending=False)
    axes[0].barh(missing_cols_sorted['Column'], missing_cols_sorted['Null Count'], color='coral')
    axes[0].set_title('Missing Values Count by Column', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Number of Missing Values', fontsize=12)
    
    axes[1].barh(missing_cols_sorted['Column'], missing_cols_sorted['Null Percentage'], color='lightblue')
    axes[1].set_title('Missing Values Percentage by Column', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Percentage (%)', fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Summary statistics by target variable
print("\n" + "=" * 80)
print("SUMMARY STATISTICS BY TARGET VARIABLE")
print("=" * 80)
for col in numerical_cols:
    print(f"\n{col}:")
    stats_by_target = df_train.groupby('SeriousDlqin2yrs')[col].agg(['mean', 'median', 'std', 'min', 'max'])
    print(stats_by_target)

ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']

## Conclusions from Exploratory Data Analysis

Based on the exploratory data analysis, the following key findings can be summarized:

### 1. **Dataset Overview**
- The training dataset contains a substantial number of observations with multiple features related to credit risk
- The dataset includes both numerical and categorical variables related to credit risk assessment
- Missing values are present in certain variables (e.g., MonthlyIncome, NumberOfDependents), which require appropriate handling

### 2. **Target Variable Distribution**
- The target variable (SeriousDlqin2yrs) shows significant class imbalance
- The majority of customers did not default (class 0), while a smaller proportion defaulted (class 1)
- The imbalance ratio indicates that default cases are relatively rare
- This class imbalance will need to be addressed during model training to ensure the model can effectively learn from both classes

### 3. **Key Variable Insights**
- **RevolvingUtilizationOfUnsecuredLines**: Shows high variability and may be a strong predictor of default risk
- **Age**: Different age groups may show different default patterns
- **DebtRatio**: Higher debt ratios are likely associated with increased default risk
- **MonthlyIncome**: Missing values are present; income levels significantly differ between default and non-default groups
- **Past Due Variables**: Variables related to past due payments (30-59, 60-89, 90+ days) show strong correlations with default risk

### 4. **Correlation Analysis**
- Several variables show moderate to strong correlations with the target variable
- Some predictor variables are correlated with each other, which may require attention during feature selection
- The correlation matrix helps identify which variables are most predictive of default

### 5. **Data Quality Issues**
- Missing values in MonthlyIncome and NumberOfDependents need to be imputed
- Some variables show extreme outliers that may need to be treated or capped
- Data distributions are often right-skewed, suggesting potential need for transformations

### 6. **Business Implications**
- The analysis reveals clear patterns that can be used to distinguish between default and non-default customers
- Variables related to credit utilization, past payment behavior, and financial capacity are particularly important
- The findings support the development of a scoring model that can effectively assess credit risk at the application stage


# 2.Creating additional variables - (Task weight: 10%)

Be creative: the more variables, the higher the score! However, variables must be logical; illogical variables will not be accepted.

- 0 points if the task is not completed.
- 0.5 points - 2 additional variables created.
- 1 point - more than 3 variables created.


In [ ]:
# Creating Additional Variables for Model Training
# The goal is to create meaningful features that can help predict credit default

# First, let's analyze missing values and decide on handling strategy
print("=" * 80)
print("MISSING VALUES ANALYSIS")
print("=" * 80)

missing_analysis = df_train[['MonthlyIncome', 'NumberOfDependents']].isnull().sum()
total_rows = len(df_train)
print(f"\nTotal rows in dataset: {total_rows:,}")
print(f"\nMissing values:")
print(f"  MonthlyIncome: {missing_analysis['MonthlyIncome']:,} ({missing_analysis['MonthlyIncome']/total_rows*100:.2f}%)")
print(f"  NumberOfDependents: {missing_analysis['NumberOfDependents']:,} ({missing_analysis['NumberOfDependents']/total_rows*100:.2f}%)")

# Check rows with any missing values
rows_with_any_missing = df_train[['MonthlyIncome', 'NumberOfDependents']].isnull().any(axis=1).sum()
print(f"\nRows with at least one missing value: {rows_with_any_missing:,} ({rows_with_any_missing/total_rows*100:.2f}%)")

# Check if missing values are related to target variable
if 'SeriousDlqin2yrs' in df_train.columns:
    missing_by_target = df_train.groupby('SeriousDlqin2yrs')[['MonthlyIncome', 'NumberOfDependents']].apply(lambda x: x.isnull().sum())
    print(f"\nMissing values by target variable:")
    print(missing_by_target)
    
    # Check if missingness is random or related to target
    missing_income_default_rate = df_train[df_train['MonthlyIncome'].isnull()]['SeriousDlqin2yrs'].mean()
    missing_income_non_missing_rate = df_train[df_train['MonthlyIncome'].notna()]['SeriousDlqin2yrs'].mean()
    print(f"\nDefault rate when MonthlyIncome is missing: {missing_income_default_rate:.4f}")
    print(f"Default rate when MonthlyIncome is not missing: {missing_income_non_missing_rate:.4f}")
    
    if abs(missing_income_default_rate - missing_income_non_missing_rate) > 0.05:
        print("⚠️  WARNING: Missing values in MonthlyIncome appear to be related to default rate!")
        print("   Deleting rows might introduce bias. Imputation is recommended.")
    else:
        print("✓ Missing values appear to be relatively random")

# Decision: We'll use imputation (median) as it preserves more data
# But we'll also show what happens if we delete rows
USE_DELETION = False  # Set to True if you want to delete rows instead of imputing

if USE_DELETION:
    print("\n" + "=" * 80)
    print("USING DELETION STRATEGY - Removing rows with missing values")
    print("=" * 80)
    df_train_features = df_train.dropna(subset=['MonthlyIncome', 'NumberOfDependents']).copy()
    df_test_features = df_test.dropna(subset=['MonthlyIncome', 'NumberOfDependents']).copy()
    print(f"Rows after deletion: {len(df_train_features):,} (lost {len(df_train) - len(df_train_features):,} rows, {((len(df_train) - len(df_train_features))/len(df_train)*100):.2f}%)")
else:
    print("\n" + "=" * 80)
    print("USING IMPUTATION STRATEGY - Filling missing values with median")
    print("=" * 80)
    print("This preserves all data and is generally recommended for credit risk modeling")
    df_train_features = df_train.copy()
    df_test_features = df_test.copy()

print("\n" + "=" * 80)
print("CREATING ADDITIONAL VARIABLES")
print("=" * 80)

# 1. Total Past Due Events - Sum of all past due occurrences
# This captures overall payment delinquency history
df_train_features['TotalPastDue'] = (
    df_train_features['NumberOfTime30-59DaysPastDueNotWorse'] + 
    df_train_features['NumberOfTime60-89DaysPastDueNotWorse'] + 
    df_train_features['NumberOfTimes90DaysLate']
)
df_test_features['TotalPastDue'] = (
    df_test_features['NumberOfTime30-59DaysPastDueNotWorse'] + 
    df_test_features['NumberOfTime60-89DaysPastDueNotWorse'] + 
    df_test_features['NumberOfTimes90DaysLate']
)
print("✓ Created: TotalPastDue - Sum of all past due occurrences")

# 2. Credit Utilization to Income Ratio
# Higher utilization relative to income may indicate financial stress
# Handle missing MonthlyIncome by using median
median_income_train = df_train_features['MonthlyIncome'].median()
median_income_test = df_test_features['MonthlyIncome'].median()

df_train_features['MonthlyIncome_filled'] = df_train_features['MonthlyIncome'].fillna(median_income_train)
df_test_features['MonthlyIncome_filled'] = df_test_features['MonthlyIncome'].fillna(median_income_test)

# Create ratio (avoid division by zero)
df_train_features['UtilizationToIncomeRatio'] = np.where(
    df_train_features['MonthlyIncome_filled'] > 0,
    df_train_features['RevolvingUtilizationOfUnsecuredLines'] / (df_train_features['MonthlyIncome_filled'] + 1),
    df_train_features['RevolvingUtilizationOfUnsecuredLines']
)
df_test_features['UtilizationToIncomeRatio'] = np.where(
    df_test_features['MonthlyIncome_filled'] > 0,
    df_test_features['RevolvingUtilizationOfUnsecuredLines'] / (df_test_features['MonthlyIncome_filled'] + 1),
    df_test_features['RevolvingUtilizationOfUnsecuredLines']
)
print("✓ Created: UtilizationToIncomeRatio - Credit utilization relative to income")

# 3. Debt Service Ratio (DebtRatio adjusted for income)
# This provides a more interpretable measure of debt burden
df_train_features['DebtServiceRatio'] = df_train_features['DebtRatio'] * df_train_features['MonthlyIncome_filled']
df_test_features['DebtServiceRatio'] = df_test_features['DebtRatio'] * df_test_features['MonthlyIncome_filled']
print("✓ Created: DebtServiceRatio - Debt burden adjusted for income")

# 4. Credit Lines Utilization Rate
# Ratio of open credit lines to total credit capacity indicator
df_train_features['CreditLinesUtilization'] = np.where(
    df_train_features['NumberOfOpenCreditLinesAndLoans'] > 0,
    df_train_features['RevolvingUtilizationOfUnsecuredLines'] / df_train_features['NumberOfOpenCreditLinesAndLoans'],
    df_train_features['RevolvingUtilizationOfUnsecuredLines']
)
df_test_features['CreditLinesUtilization'] = np.where(
    df_test_features['NumberOfOpenCreditLinesAndLoans'] > 0,
    df_test_features['RevolvingUtilizationOfUnsecuredLines'] / df_test_features['NumberOfOpenCreditLinesAndLoans'],
    df_test_features['RevolvingUtilizationOfUnsecuredLines']
)
print("✓ Created: CreditLinesUtilization - Utilization per credit line")

# 5. High Risk Past Due Indicator
# Binary indicator for customers with severe past due history (90+ days)
df_train_features['HighRiskPastDue'] = (df_train_features['NumberOfTimes90DaysLate'] > 0).astype(int)
df_test_features['HighRiskPastDue'] = (df_test_features['NumberOfTimes90DaysLate'] > 0).astype(int)
print("✓ Created: HighRiskPastDue - Indicator for severe delinquency history")

# 6. Real Estate to Total Credit Ratio
# Proportion of real estate loans in total credit portfolio
df_train_features['RealEstateRatio'] = np.where(
    df_train_features['NumberOfOpenCreditLinesAndLoans'] > 0,
    df_train_features['NumberRealEstateLoansOrLines'] / df_train_features['NumberOfOpenCreditLinesAndLoans'],
    0
)
df_test_features['RealEstateRatio'] = np.where(
    df_test_features['NumberOfOpenCreditLinesAndLoans'] > 0,
    df_test_features['NumberRealEstateLoansOrLines'] / df_test_features['NumberOfOpenCreditLinesAndLoans'],
    0
)
print("✓ Created: RealEstateRatio - Proportion of real estate loans")

# 7. Dependents Adjusted Income
# Income per dependent, which may indicate financial capacity
median_dependents_train = df_train_features['NumberOfDependents'].median()
median_dependents_test = df_test_features['NumberOfDependents'].median()

df_train_features['NumberOfDependents_filled'] = df_train_features['NumberOfDependents'].fillna(median_dependents_train)
df_test_features['NumberOfDependents_filled'] = df_test_features['NumberOfDependents'].fillna(median_dependents_test)

df_train_features['IncomePerDependent'] = np.where(
    df_train_features['NumberOfDependents_filled'] > 0,
    df_train_features['MonthlyIncome_filled'] / (df_train_features['NumberOfDependents_filled'] + 1),
    df_train_features['MonthlyIncome_filled']
)
df_test_features['IncomePerDependent'] = np.where(
    df_test_features['NumberOfDependents_filled'] > 0,
    df_test_features['MonthlyIncome_filled'] / (df_test_features['NumberOfDependents_filled'] + 1),
    df_test_features['MonthlyIncome_filled']
)
print("✓ Created: IncomePerDependent - Income capacity per dependent")

# Display summary of new variables
print("\n" + "=" * 80)
print("SUMMARY OF NEW VARIABLES")
print("=" * 80)
new_vars = ['TotalPastDue', 'UtilizationToIncomeRatio', 'DebtServiceRatio', 
            'CreditLinesUtilization', 'HighRiskPastDue', 'RealEstateRatio', 'IncomePerDependent']
print(f"\nCreated {len(new_vars)} additional variables:")
for var in new_vars:
    print(f"  - {var}")

print("\n" + "=" * 80)
print("NEW VARIABLES DESCRIPTIVE STATISTICS")
print("=" * 80)
print(df_train_features[new_vars].describe())

# Check correlation of new variables with target
print("\n" + "=" * 80)
print("CORRELATION OF NEW VARIABLES WITH TARGET")
print("=" * 80)
corr_new = df_train_features[new_vars + ['SeriousDlqin2yrs']].corr()['SeriousDlqin2yrs'].sort_values(ascending=False)
print(corr_new)

# 3. Model building - (Task weight: 50%)
A logistic regression must be built on the WoE variables.

- If any other model is built, the score is 0.

In [ ]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

# Prepare features and target from feature-engineered data (cell 10)
feature_cols = [
    'RevolvingUtilizationOfUnsecuredLines', 'age',
    'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio',
    'MonthlyIncome_filled', 'NumberOfOpenCreditLinesAndLoans',
    'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines',
    'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents_filled',
    'TotalPastDue', 'UtilizationToIncomeRatio', 'DebtServiceRatio',
    'CreditLinesUtilization', 'HighRiskPastDue', 'RealEstateRatio', 'IncomePerDependent'
]

X = df_train_features[feature_cols].copy()
y = df_train_features['SeriousDlqin2yrs'].copy()

# Split into train and validation sets (80/20 split with stratification)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("=" * 80)
print("DATA SPLIT SUMMARY")
print("=" * 80)
print(f"Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validation set: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"\nTraining target distribution:")
print(y_train.value_counts())
print(f"\nValidation target distribution:")
print(y_val.value_counts())



A WoE transformation must be calculated - maximum 3 points.

The WoE calculation must be done in two steps:

Step 1. Fine Classification (1 point). Splitting into a large number of bins (intervals).
- Typically, interval variables are divided into 20, 30, and so on intervals.
- For categorical variables, one category per group.
- Next, calculate the WoE for each group.

Step 2. Coarse Classification (2 points). Consolidating the intervals obtained in Step 1. The result should be no more than 5-10 intervals.
- Typically, groups with similar WoE values ​​are combined.
- The WoE must be monotonic, meaning that after your combination, the result must be interpretable (it is necessary to graphically demonstrate that the WoE is monotonic).


In [ ]:
# WoE (Weight of Evidence) Transformation
# Step 1: Fine Classification - Split into many bins (20 bins)
# Step 2: Coarse Classification - Combine bins to ensure monotonicity (max 5 bins)

def calculate_woe_iv(X_data, y_data, feature, n_bins=20):
    """
    Calculate WoE and IV for a feature
    Returns: grouped DataFrame with bin statistics, WoE dictionary, and IV score
    """
    # Create a copy and handle missing values
    data = pd.DataFrame({feature: X_data[feature].fillna(X_data[feature].median()), 
                         'target': y_data})
    
    # Create bins using quantiles (fine classification)
    try:
        data['bin'] = pd.qcut(data[feature], q=n_bins, duplicates='drop')
    except:
        # If quantile cut fails, use equal-width bins
        data['bin'] = pd.cut(data[feature], bins=n_bins, duplicates='drop')
    
    # Calculate statistics for each bin
    grouped = data.groupby('bin', observed=True).agg({
        'target': ['count', 'sum', 'mean']
    })
    grouped.columns = ['Total', 'Bad', 'Bad_Rate']
    grouped['Good'] = grouped['Total'] - grouped['Bad']
    
    # Calculate percentages
    total_good = grouped['Good'].sum()
    total_bad = grouped['Bad'].sum()
    
    if total_good == 0 or total_bad == 0:
        return None, None, None
    
    grouped['Good_Pct'] = grouped['Good'] / total_good
    grouped['Bad_Pct'] = grouped['Bad'] / total_bad
    
    # Calculate WoE: log(Good_Pct / Bad_Pct)
    grouped['WoE'] = np.log((grouped['Good_Pct'] + 1e-6) / (grouped['Bad_Pct'] + 1e-6))
    
    # Calculate IV (Information Value)
    grouped['IV'] = (grouped['Good_Pct'] - grouped['Bad_Pct']) * grouped['WoE']
    iv_total = grouped['IV'].sum()
    
    # Extract bin boundaries
    grouped['Bin_Min'] = grouped.index.map(lambda x: x.left if hasattr(x, 'left') else float('-inf'))
    grouped['Bin_Max'] = grouped.index.map(lambda x: x.right if hasattr(x, 'right') else float('inf'))
    
    return grouped, grouped['WoE'].to_dict(), iv_total

def coarse_classification(grouped, max_bins=5):
    """
    Combine bins to ensure monotonic WoE (coarse classification)
    Groups adjacent bins with similar WoE values
    """
    grouped_sorted = grouped.sort_values('Bin_Min').copy()
    woe_values = grouped_sorted['WoE'].values
    
    # Group bins with similar WoE values
    bins_list = []
    current_group = [0]
    
    for i in range(1, len(woe_values)):
        # Combine if WoE difference is small (< 0.1)
        if abs(woe_values[i] - woe_values[current_group[-1]]) < 0.1:
            current_group.append(i)
        else:
            bins_list.append(current_group)
            current_group = [i]
    bins_list.append(current_group)
    
    # Reduce to max_bins by merging smallest groups
    while len(bins_list) > max_bins:
        sizes = [len(b) for b in bins_list]
        min_idx = sizes.index(min(sizes))
        
        # Merge with neighbor
        if min_idx == 0:
            bins_list[0] = bins_list[0] + bins_list[1]
            bins_list.pop(1)
        elif min_idx == len(bins_list) - 1:
            bins_list[-2] = bins_list[-2] + bins_list[-1]
            bins_list.pop(-1)
        else:
            # Merge with closer neighbor (by WoE similarity)
            left_woe = grouped_sorted.iloc[bins_list[min_idx-1][-1]]['WoE']
            right_woe = grouped_sorted.iloc[bins_list[min_idx+1][0]]['WoE']
            current_woe = grouped_sorted.iloc[bins_list[min_idx][0]]['WoE']
            
            if abs(current_woe - left_woe) < abs(current_woe - right_woe):
                bins_list[min_idx-1] = bins_list[min_idx-1] + bins_list[min_idx]
            else:
                bins_list[min_idx] = bins_list[min_idx] + bins_list[min_idx+1]
                bins_list.pop(min_idx+1)
            bins_list.pop(min_idx)
    
    # Create new grouped dataframe
    coarse_groups = []
    for bin_group in bins_list:
        group_data = grouped_sorted.iloc[bin_group]
        coarse_groups.append({
            'Total': group_data['Total'].sum(),
            'Bad': group_data['Bad'].sum(),
            'Good': group_data['Good'].sum(),
            'Bad_Rate': group_data['Bad'].sum() / group_data['Total'].sum() if group_data['Total'].sum() > 0 else 0,
            'Bin_Min': group_data['Bin_Min'].min(),
            'Bin_Max': group_data['Bin_Max'].max()
        })
    
    coarse_df = pd.DataFrame(coarse_groups)
    total_good = coarse_df['Good'].sum()
    total_bad = coarse_df['Bad'].sum()
    
    # Recalculate WoE for combined bins
    coarse_df['Good_Pct'] = coarse_df['Good'] / total_good
    coarse_df['Bad_Pct'] = coarse_df['Bad'] / total_bad
    coarse_df['WoE'] = np.log((coarse_df['Good_Pct'] + 1e-6) / (coarse_df['Bad_Pct'] + 1e-6))
    coarse_df['IV'] = (coarse_df['Good_Pct'] - coarse_df['Bad_Pct']) * coarse_df['WoE']
    
    return coarse_df

# Step 1: Fine Classification
print("=" * 80)
print("STEP 1: FINE CLASSIFICATION (20 bins per feature)")
print("=" * 80)

# Select numerical features for WoE transformation
woe_features = [
    'RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio',
    'MonthlyIncome_filled', 'NumberOfOpenCreditLinesAndLoans',
    'NumberRealEstateLoansOrLines', 'NumberOfDependents_filled',
    'TotalPastDue', 'UtilizationToIncomeRatio', 'DebtServiceRatio',
    'CreditLinesUtilization', 'RealEstateRatio', 'IncomePerDependent'
]

fine_classification_results = {}
iv_scores = {}

for feature in woe_features:
    if feature in X_train.columns:
        grouped, woe_map, iv = calculate_woe_iv(X_train, y_train, feature, n_bins=20)
        if grouped is not None:
            fine_classification_results[feature] = grouped
            iv_scores[feature] = iv
            print(f"{feature:35s} - IV: {iv:.4f}, Bins: {len(grouped)}")

# Display IV scores
print("\n" + "=" * 80)
print("INFORMATION VALUE (IV) SCORES")
print("=" * 80)
iv_df = pd.DataFrame(list(iv_scores.items()), columns=['Feature', 'IV'])
iv_df = iv_df.sort_values('IV', ascending=False)
print(iv_df.to_string(index=False))

# Step 2: Coarse Classification
print("\n" + "=" * 80)
print("STEP 2: COARSE CLASSIFICATION (max 5 bins per feature)")
print("=" * 80)

coarse_classification_results = {}

for feature in woe_features:
    if feature in fine_classification_results:
        coarse_df = coarse_classification(fine_classification_results[feature], max_bins=5)
        coarse_classification_results[feature] = coarse_df
        print(f"\n{feature}: {len(coarse_df)} bins")
        print(coarse_df[['Bin_Min', 'Bin_Max', 'Total', 'Bad', 'Bad_Rate', 'WoE']].round(4))

# Visualize WoE monotonicity
print("\n" + "=" * 80)
print("VISUALIZING WoE MONOTONICITY")
print("=" * 80)

top_features = iv_df.head(6)['Feature'].tolist()
n_plots = min(len(top_features), 6)
n_cols = 3
n_rows = (n_plots + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
if n_plots == 1:
    axes = [axes]
else:
    axes = axes.flatten()

for idx, feature in enumerate(top_features[:n_plots]):
    if feature in coarse_classification_results:
        coarse_df = coarse_classification_results[feature]
        bin_centers = (coarse_df['Bin_Min'] + coarse_df['Bin_Max']) / 2
        
        axes[idx].plot(bin_centers, coarse_df['WoE'], marker='o', linewidth=2, markersize=8)
        axes[idx].set_title(f'{feature}\nWoE Monotonicity', fontsize=11, fontweight='bold')
        axes[idx].set_xlabel('Feature Value (Bin Center)', fontsize=10)
        axes[idx].set_ylabel('WoE', fontsize=10)
        axes[idx].grid(True, alpha=0.3)
        axes[idx].axhline(y=0, color='r', linestyle='--', alpha=0.5)

# Hide unused subplots
for idx in range(n_plots, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Apply WoE transformation
print("\n" + "=" * 80)
print("APPLYING WoE TRANSFORMATION TO DATA")
print("=" * 80)

def apply_woe_transformation(X_data, feature, coarse_df):
    """Apply WoE transformation to a feature using coarse classification bins"""
    feature_values = X_data[feature].fillna(X_data[feature].median())
    
    # Create bins based on coarse classification boundaries
    bins = [-np.inf] + coarse_df['Bin_Max'].tolist()[:-1] + [np.inf]
    labels = range(len(coarse_df))
    
    binned = pd.cut(feature_values, bins=bins, labels=labels, include_lowest=True)
    
    # Map to WoE values
    woe_mapping = dict(zip(labels, coarse_df['WoE'].values))
    woe_values = binned.map(woe_mapping)
    
    return woe_values

# Apply WoE to training and validation sets
X_train_woe = X_train.copy()
X_val_woe = X_val.copy()

for feature in woe_features:
    if feature in coarse_classification_results:
        X_train_woe[f'{feature}_WoE'] = apply_woe_transformation(
            X_train, feature, coarse_classification_results[feature]
        )
        X_val_woe[f'{feature}_WoE'] = apply_woe_transformation(
            X_val, feature, coarse_classification_results[feature]
        )
        print(f"✓ {feature}")

# Prepare final feature sets for modeling
woe_feature_cols = [f'{f}_WoE' for f in woe_features if f in coarse_classification_results]
binary_features = ['HighRiskPastDue', 'NumberOfTime30-59DaysPastDueNotWorse', 
                   'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate']

# Select only features that exist in the data
available_binary = [f for f in binary_features if f in X_train_woe.columns]

X_train_final = X_train_woe[woe_feature_cols + available_binary].copy()
X_val_final = X_val_woe[woe_feature_cols + available_binary].copy()

print(f"\nFinal feature set: {len(X_train_final.columns)} features")
print(f"Features: {X_train_final.columns.tolist()}")

Constructing and evaluating logistic regression - maximum 1 point
- Constructing only logistic regression - 0.3 points
- Evaluating the model (roc auc, f1, etc.) - 0.3 points
- Constructing a scorecard - 0.4 points

As a reminder, the following formulas are required for the scorecard (details in the lecture and seminar):
Score_i =  (βi × WoE_i + α/n) × Factor + Offset/n, где

- Factor = pdo/ln(2)

- Offset = Target Score — (Factor × ln(Target Odds))

In [ ]:
# Building Logistic Regression Model on WoE Variables
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_score, recall_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("LOGISTIC REGRESSION MODEL")
print("=" * 80)

# Handle any remaining missing values
X_train_final = X_train_final.fillna(0)
X_val_final = X_val_final.fillna(0)

# Build logistic regression model
lr_model = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')
lr_model.fit(X_train_final, y_train)

print(f"✓ Model trained with {len(X_train_final.columns)} features")

# Make predictions
y_train_pred_proba = lr_model.predict_proba(X_train_final)[:, 1]
y_val_pred_proba = lr_model.predict_proba(X_val_final)[:, 1]
y_train_pred = lr_model.predict(X_train_final)
y_val_pred = lr_model.predict(X_val_final)

# Model Evaluation
print("\n" + "=" * 80)
print("MODEL EVALUATION")
print("=" * 80)

# ROC AUC Score
train_auc = roc_auc_score(y_train, y_train_pred_proba)
val_auc = roc_auc_score(y_val, y_val_pred_proba)

print(f"\nROC AUC Score:")
print(f"  Training: {train_auc:.4f}")
print(f"  Validation: {val_auc:.4f}")

# F1 Score
train_f1 = f1_score(y_train, y_train_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"\nF1 Score:")
print(f"  Training: {train_f1:.4f}")
print(f"  Validation: {val_f1:.4f}")

# Precision and Recall
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
val_precision = precision_score(y_val, y_val_pred)
val_recall = recall_score(y_val, y_val_pred)

print(f"\nPrecision:")
print(f"  Training: {train_precision:.4f}")
print(f"  Validation: {val_precision:.4f}")

print(f"\nRecall:")
print(f"  Training: {train_recall:.4f}")
print(f"  Validation: {val_recall:.4f}")

# Confusion Matrix
print(f"\nConfusion Matrix (Validation):")
cm_val = confusion_matrix(y_val, y_val_pred)
print(cm_val)
print(f"\nTrue Negatives: {cm_val[0,0]}, False Positives: {cm_val[0,1]}")
print(f"False Negatives: {cm_val[1,0]}, True Positives: {cm_val[1,1]}")

# Classification Report
print(f"\nClassification Report (Validation):")
print(classification_report(y_val, y_val_pred, target_names=['No Default', 'Default']))

# ROC Curve
fpr_train, tpr_train, _ = roc_curve(y_train, y_train_pred_proba)
fpr_val, tpr_val, _ = roc_curve(y_val, y_val_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr_train, tpr_train, label=f'Training ROC (AUC = {train_auc:.4f})', linewidth=2)
plt.plot(fpr_val, tpr_val, label=f'Validation ROC (AUC = {val_auc:.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Logistic Regression', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Scorecard Construction
print("\n" + "=" * 80)
print("SCORECARD CONSTRUCTION")
print("=" * 80)

# Scorecard parameters
pdo = 20  # Points to Double Odds (standard: 20 points doubles the odds)
target_score = 600  # Target score at target odds
target_odds = 50  # Target odds (e.g., 50:1 means 1 default per 50 good)

# Calculate Factor and Offset
factor = pdo / np.log(2)
offset = target_score - (factor * np.log(target_odds))

print(f"Scorecard Parameters:")
print(f"  PDO (Points to Double Odds): {pdo}")
print(f"  Target Score: {target_score}")
print(f"  Target Odds: {target_odds}:1")
print(f"  Factor: {factor:.4f}")
print(f"  Offset: {offset:.4f}")

# Get model coefficients
coefficients = lr_model.coef_[0]
intercept = lr_model.intercept_[0]
n_features = len(coefficients)

print(f"\nModel Intercept (α): {intercept:.4f}")
print(f"Number of features (n): {n_features}")

# Calculate score for each feature
scorecard = pd.DataFrame({
    'Feature': X_train_final.columns,
    'Coefficient': coefficients,
    'Factor_Component': coefficients * factor,
    'Offset_Component': offset / n_features
})

scorecard['Points'] = scorecard['Factor_Component'] + scorecard['Offset_Component']
scorecard = scorecard.sort_values('Points', ascending=False)

print(f"\nScorecard (Points per feature):")
print(scorecard[['Feature', 'Coefficient', 'Points']].to_string(index=False))

# Calculate base score (from intercept)
base_score = (intercept / n_features) * factor + (offset / n_features)
print(f"\nBase Score (from intercept): {base_score:.2f}")

# Example: Calculate score for a sample observation
sample_idx = 0
sample_woe = X_train_final.iloc[sample_idx]
sample_score = base_score + sample_woe.dot(scorecard.set_index('Feature')['Points'])
sample_prob = lr_model.predict_proba(X_train_final.iloc[sample_idx:sample_idx+1])[0, 1]

print(f"\nExample Score Calculation:")
print(f"  Sample observation index: {sample_idx}")
print(f"  Predicted probability: {sample_prob:.4f}")
print(f"  Calculated score: {sample_score:.2f}")

# Score distribution
train_scores = base_score + X_train_final.dot(scorecard.set_index('Feature')['Points'])
val_scores = base_score + X_val_final.dot(scorecard.set_index('Feature')['Points'])

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(train_scores[y_train == 0], bins=50, alpha=0.7, label='No Default', color='skyblue', edgecolor='black')
plt.hist(train_scores[y_train == 1], bins=50, alpha=0.7, label='Default', color='salmon', edgecolor='black')
plt.xlabel('Score', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Score Distribution - Training Set', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(val_scores[y_val == 0], bins=50, alpha=0.7, label='No Default', color='skyblue', edgecolor='black')
plt.hist(val_scores[y_val == 1], bins=50, alpha=0.7, label='Default', color='salmon', edgecolor='black')
plt.xlabel('Score', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Score Distribution - Validation Set', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nScore Statistics:")
print(f"  Training - Mean: {train_scores.mean():.2f}, Std: {train_scores.std():.2f}")
print(f"  Training - Min: {train_scores.min():.2f}, Max: {train_scores.max():.2f}")
print(f"  Validation - Mean: {val_scores.mean():.2f}, Std: {val_scores.std():.2f}")
print(f"  Validation - Min: {val_scores.min():.2f}, Max: {val_scores.max():.2f}")

In [ ]:
# Population Stability Index (PSI) Analysis
# PSI measures the stability of variable distributions between train and test sets

def calculate_psi(expected, actual, bins=10):
    """
    Calculate Population Stability Index (PSI)
    PSI < 0.1: No significant change
    0.1 <= PSI < 0.25: Moderate change
    PSI >= 0.25: Significant change
    """
    # Create bins based on expected distribution
    breakpoints = np.linspace(expected.min(), expected.max(), bins + 1)
    breakpoints[0] = -np.inf
    breakpoints[-1] = np.inf
    
    # Calculate distributions
    expected_pct = pd.cut(expected, breakpoints).value_counts(normalize=True, sort=False)
    actual_pct = pd.cut(actual, breakpoints).value_counts(normalize=True, sort=False)
    
    # Handle zero values
    expected_pct = expected_pct + 1e-6
    actual_pct = actual_pct + 1e-6
    
    # Calculate PSI
    psi = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
    return psi

print("=" * 80)
print("POPULATION STABILITY INDEX (PSI) ANALYSIS")
print("=" * 80)
print("Comparing df_train_features (expected) vs df_test_features (actual)")

# Use features from feature engineering (cell 10)
# Select numerical features for PSI calculation
psi_features = [
    'RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio',
    'MonthlyIncome_filled', 'NumberOfOpenCreditLinesAndLoans',
    'NumberRealEstateLoansOrLines', 'NumberOfDependents_filled',
    'TotalPastDue', 'UtilizationToIncomeRatio', 'DebtServiceRatio',
    'CreditLinesUtilization', 'RealEstateRatio', 'IncomePerDependent'
]

psi_results = {}

print("\n" + "=" * 80)
print("PSI RESULTS BY FEATURE")
print("=" * 80)

for feature in psi_features:
    if feature in df_train_features.columns and feature in df_test_features.columns:
        # Handle missing values
        train_vals = df_train_features[feature].fillna(df_train_features[feature].median())
        test_vals = df_test_features[feature].fillna(df_test_features[feature].median())
        
        # Calculate PSI
        psi = calculate_psi(train_vals, test_vals, bins=10)
        psi_results[feature] = psi
        
        # Interpret PSI
        if psi < 0.1:
            interpretation = "No significant change"
        elif psi < 0.25:
            interpretation = "Moderate change"
        else:
            interpretation = "Significant change"
        
        print(f"{feature:35s} PSI: {psi:7.4f} - {interpretation}")

# Create summary DataFrame
psi_df = pd.DataFrame(list(psi_results.items()), columns=['Feature', 'PSI'])
psi_df = psi_df.sort_values('PSI', ascending=False)

# Categorize PSI values
psi_df['Stability'] = psi_df['PSI'].apply(lambda x: 
    'No change' if x < 0.1 else ('Moderate change' if x < 0.25 else 'Significant change'))

print("\n" + "=" * 80)
print("PSI SUMMARY")
print("=" * 80)
print(psi_df.to_string(index=False))

# Visualize PSI results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# PSI values bar chart
axes[0].barh(range(len(psi_df)), psi_df['PSI'], color=['green' if x < 0.1 else ('orange' if x < 0.25 else 'red') 
                                                         for x in psi_df['PSI']])
axes[0].set_yticks(range(len(psi_df)))
axes[0].set_yticklabels(psi_df['Feature'], fontsize=9)
axes[0].set_xlabel('PSI Value', fontsize=12)
axes[0].set_title('PSI Values by Feature', fontsize=14, fontweight='bold')
axes[0].axvline(x=0.1, color='orange', linestyle='--', label='Moderate threshold')
axes[0].axvline(x=0.25, color='red', linestyle='--', label='Significant threshold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='x')

# Stability categories
stability_counts = psi_df['Stability'].value_counts()
axes[1].bar(stability_counts.index, stability_counts.values, 
            color=['green', 'orange', 'red'][:len(stability_counts)])
axes[1].set_ylabel('Number of Features', fontsize=12)
axes[1].set_title('Stability Categories', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(stability_counts.values):
    axes[1].text(i, v + 0.5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Overall assessment
high_psi_features = psi_df[psi_df['PSI'] >= 0.25]
moderate_psi_features = psi_df[(psi_df['PSI'] >= 0.1) & (psi_df['PSI'] < 0.25)]

print("\n" + "=" * 80)
print("OVERALL ASSESSMENT")
print("=" * 80)
print(f"Features with significant change (PSI >= 0.25): {len(high_psi_features)}")
if len(high_psi_features) > 0:
    print("  " + ", ".join(high_psi_features['Feature'].tolist()))

print(f"\nFeatures with moderate change (0.1 <= PSI < 0.25): {len(moderate_psi_features)}")
if len(moderate_psi_features) > 0:
    print("  " + ", ".join(moderate_psi_features['Feature'].tolist()))

print(f"\nFeatures with no significant change (PSI < 0.1): {len(psi_df[psi_df['PSI'] < 0.1])}")

if len(high_psi_features) > 0:
    print("\n⚠️  WARNING: Some features show significant distribution changes between train and test sets.")
    print("   This may indicate data drift and could affect model performance.")
else:
    print("\n✓ Overall, the distributions are relatively stable between train and test sets.")


Conduct a sample stability analysis using PSI
- Compare the test and training samples you downloaded from Kaggle (df_train, df_test)

# 4. Using methods to reduce class imbalance - (Task weight: 20%)
- Try several methods to reduce class imbalance
- Choose the one that brings the greatest improvement

In [ ]:
# Methods to Reduce Class Imbalance
# Try several methods and compare their performance

from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

print("=" * 80)
print("CLASS IMBALANCE REDUCTION METHODS")
print("=" * 80)

# Original class distribution
print("\nOriginal class distribution:")
print(f"  Class 0 (No Default): {Counter(y_train)[0]}")
print(f"  Class 1 (Default): {Counter(y_train)[1]}")
print(f"  Imbalance Ratio: {Counter(y_train)[0] / Counter(y_train)[1]:.2f}:1")

# Prepare data for resampling
X_train_resample = X_train_final.fillna(0).copy()
y_train_resample = y_train.copy()
X_val_resample = X_val_final.fillna(0).copy()

# Method 1: Random Over-Sampling
print("\n" + "=" * 80)
print("METHOD 1: Random Over-Sampling")
print("=" * 80)
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train_resample, y_train_resample)
print(f"After resampling - Class 0: {Counter(y_ros)[0]}, Class 1: {Counter(y_ros)[1]}")

# Method 2: SMOTE (Synthetic Minority Over-sampling Technique)
print("\n" + "=" * 80)
print("METHOD 2: SMOTE")
print("=" * 80)
try:
    smote = SMOTE(random_state=42, k_neighbors=5)
    X_smote, y_smote = smote.fit_resample(X_train_resample, y_train_resample)
    print(f"After resampling - Class 0: {Counter(y_smote)[0]}, Class 1: {Counter(y_smote)[1]}")
except Exception as e:
    print(f"SMOTE failed: {e}")
    X_smote, y_smote = X_train_resample, y_train_resample

# Method 3: ADASYN (Adaptive Synthetic Sampling)
print("\n" + "=" * 80)
print("METHOD 3: ADASYN")
print("=" * 80)
try:
    adasyn = ADASYN(random_state=42, n_neighbors=5)
    X_adasyn, y_adasyn = adasyn.fit_resample(X_train_resample, y_train_resample)
    print(f"After resampling - Class 0: {Counter(y_adasyn)[0]}, Class 1: {Counter(y_adasyn)[1]}")
except Exception as e:
    print(f"ADASYN failed: {e}")
    X_adasyn, y_adasyn = X_train_resample, y_train_resample

# Method 4: Random Under-Sampling
print("\n" + "=" * 80)
print("METHOD 4: Random Under-Sampling")
print("=" * 80)
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train_resample, y_train_resample)
print(f"After resampling - Class 0: {Counter(y_rus)[0]}, Class 1: {Counter(y_rus)[1]}")

# Method 5: SMOTE + Tomek Links (Combination)
print("\n" + "=" * 80)
print("METHOD 5: SMOTE + Tomek Links")
print("=" * 80)
try:
    smote_tomek = SMOTETomek(random_state=42)
    X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X_train_resample, y_train_resample)
    print(f"After resampling - Class 0: {Counter(y_smote_tomek)[0]}, Class 1: {Counter(y_smote_tomek)[1]}")
except Exception as e:
    print(f"SMOTE+Tomek failed: {e}")
    X_smote_tomek, y_smote_tomek = X_train_resample, y_train_resample

# Method 6: Class Weight Balancing (in model)
print("\n" + "=" * 80)
print("METHOD 6: Class Weight Balancing (in Logistic Regression)")
print("=" * 80)
# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"Class weights: {class_weight_dict}")

# Train and evaluate models with different resampling methods
print("\n" + "=" * 80)
print("TRAINING AND EVALUATING MODELS")
print("=" * 80)

methods = {
    'Original (No Resampling)': (X_train_resample, y_train_resample, False),
    'Random Over-Sampling': (X_ros, y_ros, False),
    'SMOTE': (X_smote, y_smote, False),
    'ADASYN': (X_adasyn, y_adasyn, False),
    'Random Under-Sampling': (X_rus, y_rus, False),
    'SMOTE + Tomek': (X_smote_tomek, y_smote_tomek, False),
    'Class Weight Balancing': (X_train_resample, y_train_resample, True)
}

results = []

for method_name, (X_method, y_method, use_class_weight) in methods.items():
    print(f"\n{method_name}:")
    
    # Train model
    if use_class_weight:
        lr_method = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs', 
                                       class_weight='balanced')
    else:
        lr_method = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')
    
    lr_method.fit(X_method, y_method)
    
    # Predictions
    y_val_pred_proba = lr_method.predict_proba(X_val_resample)[:, 1]
    y_val_pred = lr_method.predict(X_val_resample)
    
    # Calculate metrics
    metrics = {
        'Method': method_name,
        'AUC': roc_auc_score(y_val, y_val_pred_proba),
        'F1': f1_score(y_val, y_val_pred),
        'Precision': precision_score(y_val, y_val_pred),
        'Recall': recall_score(y_val, y_val_pred)
    }
    results.append(metrics)
    
    print(f"  AUC: {metrics['AUC']:.4f}, F1: {metrics['F1']:.4f}, "
          f"Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}")

# Compare results
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('AUC', ascending=False)

print("\n" + "=" * 80)
print("COMPARISON OF METHODS")
print("=" * 80)
print(results_df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# AUC Comparison
axes[0, 0].barh(results_df['Method'], results_df['AUC'], color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('ROC AUC Score', fontsize=12)
axes[0, 0].set_title('ROC AUC Comparison', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='x')
axes[0, 0].axvline(x=results_df['AUC'].max(), color='red', linestyle='--', alpha=0.7)

# F1 Score Comparison
axes[0, 1].barh(results_df['Method'], results_df['F1'], color='lightgreen', edgecolor='black')
axes[0, 1].set_xlabel('F1 Score', fontsize=12)
axes[0, 1].set_title('F1 Score Comparison', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='x')
axes[0, 1].axvline(x=results_df['F1'].max(), color='red', linestyle='--', alpha=0.7)

# Precision-Recall Comparison
axes[1, 0].scatter(results_df['Recall'], results_df['Precision'], s=200, alpha=0.6, edgecolors='black')
for idx, row in results_df.iterrows():
    axes[1, 0].annotate(row['Method'], (row['Recall'], row['Precision']), 
                       fontsize=8, ha='center')
axes[1, 0].set_xlabel('Recall', fontsize=12)
axes[1, 0].set_ylabel('Precision', fontsize=12)
axes[1, 0].set_title('Precision-Recall Trade-off', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Combined Metrics
x_pos = np.arange(len(results_df))
width = 0.25
axes[1, 1].bar(x_pos - width, results_df['AUC'], width, label='AUC', alpha=0.8, edgecolor='black')
axes[1, 1].bar(x_pos, results_df['F1'], width, label='F1', alpha=0.8, edgecolor='black')
axes[1, 1].bar(x_pos + width, results_df['Precision'], width, label='Precision', alpha=0.8, edgecolor='black')
axes[1, 1].set_xlabel('Method', fontsize=12)
axes[1, 1].set_ylabel('Score', fontsize=12)
axes[1, 1].set_title('Combined Metrics Comparison', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(results_df['Method'], rotation=45, ha='right', fontsize=8)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Select best method
best_method = results_df.iloc[0]['Method']
best_auc = results_df.iloc[0]['AUC']
best_f1 = results_df.iloc[0]['F1']

print("\n" + "=" * 80)
print("BEST METHOD SELECTION")
print("=" * 80)
print(f"Best method based on AUC: {best_method}")
print(f"  AUC: {best_auc:.4f}")
print(f"  F1 Score: {best_f1:.4f}")
print(f"  Precision: {results_df.iloc[0]['Precision']:.4f}")
print(f"  Recall: {results_df.iloc[0]['Recall']:.4f}")

# Train final model with best method
print("\n" + "=" * 80)
print("FINAL MODEL WITH BEST METHOD")
print("=" * 80)

X_best, y_best, use_class_weight = methods[best_method]

if use_class_weight:
    final_model = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs', 
                                     class_weight='balanced')
else:
    final_model = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')

final_model.fit(X_best, y_best)

# Final predictions
y_val_final_proba = final_model.predict_proba(X_val_resample)[:, 1]
y_val_final_pred = final_model.predict(X_val_resample)

# Final metrics
final_auc = roc_auc_score(y_val, y_val_final_proba)
final_f1 = f1_score(y_val, y_val_final_pred)
final_precision = precision_score(y_val, y_val_final_pred)
final_recall = recall_score(y_val, y_val_final_pred)

print(f"\nFinal Model Performance:")
print(f"  AUC: {final_auc:.4f}")
print(f"  F1 Score: {final_f1:.4f}")
print(f"  Precision: {final_precision:.4f}")
print(f"  Recall: {final_recall:.4f}")

# Compare with original model (from cell 16)
print(f"\nImprovement over original model:")
print(f"  AUC improvement: {final_auc - val_auc:+.4f}")
print(f"  F1 improvement: {final_f1 - val_f1:+.4f}")
print(f"  Precision improvement: {final_precision - val_precision:+.4f}")
print(f"  Recall improvement: {final_recall - val_recall:+.4f}")

# ROC Curve comparison
fpr_final, tpr_final, _ = roc_curve(y_val, y_val_final_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr_val, tpr_val, label=f'Original Model (AUC = {val_auc:.4f})', 
         linewidth=2, linestyle='--', alpha=0.7)
plt.plot(fpr_final, tpr_final, label=f'Best: {best_method} (AUC = {final_auc:.4f})', 
         linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random', alpha=0.5)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'imblearn'

_Optional, for those who have reached the end of the laptop_ 😊

What was your impression of the work?
What was difficult?
What was interesting?